## Analisis de Datos: Tendencia de Victimas a Nivel Nacional a 1 año por Secuestros

De acuerdo a los datos abiertos de la fiscalia se pueden agrupar la cantidad de victimas por año de hecho referentes al delito asociado que en nuestro caso es el secuestro y desapariciones

El objetivo de este notebook es entrenar una red neuronal para encontrar la tendencia y predecir 5 años a futuro el numero de victimas por año

### Cantidad de Victimas por Año

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# Conteo de Víctimas por año y municipio
dfSecuestros = pd.read_csv('ConteoVictimasSecuestros.csv')
# Conteo de victimas por año de hecho
dfSecuestros[['ANIO_HECHO','TOTAL_VICTIMAS']].groupby('ANIO_HECHO').sum()

,TOTAL_VICTIMAS
ANIO_HECHO,
1982,24
1983,1
1984,1
1985,3
1986,11
1987,45
1988,46
1989,42
1990,358


### DataFrame 5 a 1 años

En este dataframe se representa lo siguiente:
- Input_0 -> Input_4 = Victimas por año en 5 años (ej: 1982 a 1986)
- Output_0 = Victimas en el año siguiente (ej: 1987)

In [3]:
# Conteo de Víctimas por año y municipio
dfVictimasAnios = pd.read_csv('DataSetVictimasAnios.csv')
dfVictimasAnios

,Input_0,Input_1,Input_2,Input_3,Input_4,Output_0,Output_1,Output_2,Output_3,Output_4
0,24,1,1,3,11,45,46,42,358,92
1,1,1,3,11,45,46,42,358,92,171
2,1,3,11,45,46,42,358,92,171,101
3,3,11,45,46,42,358,92,171,101,187
4,11,45,46,42,358,92,171,101,187,209
5,45,46,42,358,92,171,101,187,209,430
6,46,42,358,92,171,101,187,209,430,730
7,42,358,92,171,101,187,209,430,730,1293
8,358,92,171,101,187,209,430,730,1293,2068
9,92,171,101,187,209,430,730,1293,2068,2203


### Dividir el DataFrame en los conjuntos de prueba, validación y test

In [4]:
inputs = ['Input_{}'.format(i) for i in range(5)]
target_column = ['Output_0'] 

#predictors = list(set(list(df.columns))-set(target_column))
#df[predictors] = df[predictors]/df[predictors].max()
#df.describe()

X = dfVictimasAnios[inputs]
y = dfVictimasAnios['Output_0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(X_train.shape); print(X_test.shape)

(21, 5)
(9, 5)


### DataFrame a DataSet de Tensorflow

In [5]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(X,Y, shuffle=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((dict(X), Y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X))
    ds = ds.batch(batch_size)
    return ds

In [6]:
# Tamaño del numero de datos por iteración que se necesitan para entrenar y probar el algoritmo
batch_size = 4
train_ds = df_to_dataset(X_train,y_train, batch_size=batch_size)
test_ds = df_to_dataset(X_test,y_test, shuffle=False, batch_size=batch_size)

### Construyendo el Modelo

In [7]:
feature_columns = []

# Columnas Numericas
numericas = ['Input_{}'.format(i) for i in range(5)]
for header in numericas:
    feature_columns.append(feature_column.numeric_column(header))
    
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [8]:
model = tf.keras.Sequential([
    feature_layer,
    #layers.Dense(1, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',loss=losses.mean_squared_error,metrics=['mean_squared_error'])

model.fit(train_ds,epochs=20)
#model.summary()

Train for 6 steps
Epoch 1/20
6/6 [==============================] - 0s 33ms/step - loss: 974045.6425 - mean_squared_error: 1090770.5000
Epoch 2/20
6/6 [==============================] - 0s 3ms/step - loss: 956858.8457 - mean_squared_error: 1028428.3125
Epoch 3/20
6/6 [==============================] - 0s 3ms/step - loss: 877007.6083 - mean_squared_error: 975011.1250
Epoch 4/20
6/6 [==============================] - 0s 3ms/step - loss: 830206.5254 - mean_squared_error: 926595.6250
Epoch 5/20
6/6 [==============================] - 0s 3ms/step - loss: 812430.4401 - mean_squared_error: 891740.6875
Epoch 6/20
6/6 [==============================] - 0s 3ms/step - loss: 1007498.0276 - mean_squared_error: 856039.8750
Epoch 7/20
6/6 [==============================] - 0s 2ms/step - loss: 977884.4024 - mean_squared_error: 833473.6875
Epoch 8/20
6/6 [==============================] - 0s 3ms/step - loss: 727247.2201 - mean_squared_error: 808576.8750
Epoch 9/20
6/6 [==============================] - 

In [9]:
pred= model.predict(test_ds)
print(np.sqrt(mean_squared_error(y_test,pred)))

694.4864205011802
